In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import re

In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [27]:
df = pd.read_csv('RAIS_vinculo_poder_nivel_raca_sexo.csv', sep=';')
df

,esfera,poder,raca,genero,ano,valores
0,Estadual,Executivo,Amarela,Feminino,2004,15113.0
1,Estadual,Executivo,Amarela,Masculino,2004,10887.0
2,Estadual,Executivo,Branca,Feminino,2004,1239279.0
3,Estadual,Executivo,Branca,Masculino,2004,795102.0
4,Estadual,Executivo,Indígena,Feminino,2004,15046.0
...,...,...,...,...,...,...
1355,Municipal,Legislativo,Indígena,Masculino,2020,321.0
1356,Municipal,Legislativo,Parda,Feminino,2020,21292.0
1357,Municipal,Legislativo,Parda,Masculino,2020,25679.0
1358,Municipal,Legislativo,Preta,Feminino,2020,1953.0


In [28]:
df['raca'].unique()

array(['Amarela', 'Branca', 'Indígena', 'Parda', 'Preta'], dtype=object)

In [29]:
df['genero'].unique()

array(['Feminino', 'Masculino'], dtype=object)

In [30]:
df = df.rename(columns={"esfera" : "esfera_governo", "poder" : "poderes", "raca" :"cor_raca", "valores" : "quantidade_vinculos"})
df

,esfera_governo,poderes,cor_raca,genero,ano,quantidade_vinculos
0,Estadual,Executivo,Amarela,Feminino,2004,15113.0
1,Estadual,Executivo,Amarela,Masculino,2004,10887.0
2,Estadual,Executivo,Branca,Feminino,2004,1239279.0
3,Estadual,Executivo,Branca,Masculino,2004,795102.0
4,Estadual,Executivo,Indígena,Feminino,2004,15046.0
...,...,...,...,...,...,...
1355,Municipal,Legislativo,Indígena,Masculino,2020,321.0
1356,Municipal,Legislativo,Parda,Feminino,2020,21292.0
1357,Municipal,Legislativo,Parda,Masculino,2020,25679.0
1358,Municipal,Legislativo,Preta,Feminino,2020,1953.0


In [31]:
def transformar(nome):
    nome = re.sub(r"\bFeminino\b", "Mulheres", nome)
    nome =re.sub(r"\bMasculino\b", "Homens", nome)
    return nome

In [32]:
df['genero'] = df['genero'].apply(transformar)

In [33]:
df['genero_raca'] = df['genero'] + ' ' + df['cor_raca']

In [34]:
df['genero_raca'].unique()

array(['Mulheres Amarela', 'Homens Amarela', 'Mulheres Branca',
       'Homens Branca', 'Mulheres Indígena', 'Homens Indígena',
       'Mulheres Parda', 'Homens Parda', 'Mulheres Preta', 'Homens Preta'],
      dtype=object)

In [35]:
def transforme(nome):
    nome =re.sub(r"\bMulheres Amarela\b", "Mulheres Amarelas", nome)
    nome =re.sub(r"\bHomens Amarela\b", "Homens Amarelos", nome)
    nome =re.sub(r"\bMulheres Branca\b", "Mulheres Brancas", nome)
    nome =re.sub(r"\bHomens Branca\b", "Homens Brancos", nome) 
    nome =re.sub(r"\bMulheres Indígena\b", "Mulheres Indígenas", nome)
    nome =re.sub(r"\bHomens Indígena\b", "Homens Indígenas", nome)
    nome =re.sub(r"\bMulheres Parda\b", "Mulheres Pardas", nome)
    nome =re.sub(r"\bHomens Parda\b", "Homens Pardos", nome)
    nome =re.sub(r"\bMulheres Preta\b", "Mulheres Pretas", nome)
    nome =re.sub(r"\bHomens Preta\b", "Homens Pretos", nome)
    
    return nome

In [36]:
df['genero_raca'] = df['genero_raca'].apply(transforme)

In [37]:
df['genero_raca'].unique()

array(['Mulheres Amarelas', 'Homens Amarelos', 'Mulheres Brancas',
       'Homens Brancos', 'Mulheres Indígenas', 'Homens Indígenas',
       'Mulheres Pardas', 'Homens Pardos', 'Mulheres Pretas',
       'Homens Pretos'], dtype=object)

In [39]:
df = df[['ano', 'esfera_governo', 'poderes', 'genero', 'cor_raca', 'genero_raca', 'quantidade_vinculos']]

In [40]:
df

,ano,esfera_governo,poderes,genero,cor_raca,genero_raca,quantidade_vinculos
0,2004,Estadual,Executivo,Mulheres,Amarela,Mulheres Amarelas,15113.0
1,2004,Estadual,Executivo,Homens,Amarela,Homens Amarelos,10887.0
2,2004,Estadual,Executivo,Mulheres,Branca,Mulheres Brancas,1239279.0
3,2004,Estadual,Executivo,Homens,Branca,Homens Brancos,795102.0
4,2004,Estadual,Executivo,Mulheres,Indígena,Mulheres Indígenas,15046.0
...,...,...,...,...,...,...,...
1355,2020,Municipal,Legislativo,Homens,Indígena,Homens Indígenas,321.0
1356,2020,Municipal,Legislativo,Mulheres,Parda,Mulheres Pardas,21292.0
1357,2020,Municipal,Legislativo,Homens,Parda,Homens Pardos,25679.0
1358,2020,Municipal,Legislativo,Mulheres,Preta,Mulheres Pretas,1953.0


Subindo para o GBQ

In [41]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [42]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('esfera_governo', 'STRING', description= 'Nível da esfera do governo referente da observação'),
          bigquery.SchemaField('poderes', 'STRING', description= 'Poder abrangente ao nível de esfera referente a observação'),
          bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),
          bigquery.SchemaField('cor_raca', 'STRING', description= 'Raça/cor da pessoa observada'),
          bigquery.SchemaField('genero_raca', 'STRING', description= 'Gênero e raça da pessoa observada'),
          bigquery.SchemaField('quantidade_vinculos', 'FLOAT', description= 'Número total de vinculos observados')]

In [43]:
table_ref = dataset_ref.table('RAIS_vinculo_poder_nivel_raca_sexo')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=37df400b-d03a-4ffc-b5a0-1c2193279598>